## One-Shot Learning with Cosine Similarity
This notebook contains code for Face recognition.

Reference:

https://sefiks.com/2018/08/06/deep-face-recognition-with-keras/

https://www.robots.ox.ac.uk/~vgg/data/vgg_face/

Udemy Course:

https://www.udemy.com/share/10381KBEoacl5STXU=/


Note:
Please put the person images in people_images folder.


In [1]:
#Import modules.
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Convolution2D, ZeroPadding2D, MaxPooling2D, Flatten, Dense, Dropout, Activation
from PIL import Image
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import load_img, save_img, img_to_array
from tensorflow.keras.applications.imagenet_utils import preprocess_input
#VGG 16 CNN model.
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.utils import plot_model
import matplotlib.pyplot as plt

import cv2 as cv
import os
from os import listdir
from os.path import isfile, join

In [2]:
#VGGFace.
#https://www.robots.ox.ac.uk/~vgg/
model = Sequential()
model.add(ZeroPadding2D((1,1),input_shape=(224,224, 3)))
model.add(Convolution2D(64, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(128, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(Convolution2D(4096, (7, 7), activation='relu'))
model.add(Dropout(0.5))
model.add(Convolution2D(4096, (1, 1), activation='relu'))
model.add(Dropout(0.5))
model.add(Convolution2D(2622, (1, 1)))
model.add(Flatten())
model.add(Activation('softmax'))
#print(model.summary())

In [3]:
# Load the model.
# Download the model from https://drive.google.com/file/d/1CPSeum3HpopfomUEK1gybeuIVoeJT_Eo/view?usp=sharing
#plot_model(model, to_file='VGG-16.png')
model.load_weights('models/vgg_face_weights.h5')

In [4]:
#Function to pre-process images
def preprocess_image(image_path):
    img = load_img(image_path, target_size=(224, 224))
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = preprocess_input(img)
    return img


In [5]:
#Function to find Cosine Similarity.

def cosine_similarity(src_vector, test_vector):
    #Cosine Distance
    num=np.matmul(np.transpose(src_vector),test_vector)
    norm1=np.sum(np.multiply(src_vector,src_vector))
    norm2=np.sum(np.multiply(test_vector,test_vector))
    return 1 -(num/(np.sqrt(norm1)*np.sqrt(norm2)))

In [6]:
#Previous layer from output.
vgg_face_descriptor= Model(inputs=model.layers[0].input, outputs=model.layers[-2].output)

In [7]:
#Detect faces via camera.

#Find faces from images and save them.
def makedir(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)
        return None
    else:
        pass
#Haar Cascade Classifier.

face_detector=cv.CascadeClassifier('Haarcascades/haarcascade_frontalface_default.xml')

mypath = "./people_images/"
image_file_names = [f for f in listdir(mypath) if isfile(join(mypath, f))]
print("Collected " + str(len(image_file_names)) + " images")
makedir("./faces/")

for image_name in image_file_names:
    person_image = cv.imread(mypath+image_name)
    face_info = face_detector.detectMultiScale(person_image, 1.3, 5)
    print('Found a face')
    for (x,y,w,h) in face_info:
        face = person_image[y:y+h, x:x+w]
        roi = cv.resize(face, (128, 128), interpolation = cv.INTER_CUBIC)
        path = "./faces/" + "face_" + image_name 
        cv.imwrite(path, roi)

Collected 5 images
Found a face
Found a face
Found a face
Found a face
Found a face


In [8]:
face_folder='./faces'
#Dictionary to store feature vectors.
face_dict=dict()

for file in listdir(face_folder):
    person,ext=file.split(".")
    face_dict[person]=vgg_face_descriptor.predict(preprocess_image('./faces/%s.jpg' % (person)))[0,:]


In [ ]:
capture=cv.VideoCapture(0)

while (True):
    ret, img=capture.read()
    #Haar Cascade Classifier
    faces=face_detector.detectMultiScale(img,1.3,5)
  
    for (x,y,w,h) in faces:
        if w >100:
            #Draw bounding rectangle.
            cv.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
            #Crop the image.
            face_detect=img[int(y):int(y+h),int(x):int(x+w)]
            #Resize to VGGFace dimensions.
            face_detect=cv.resize(face_detect,(224,224))
            #PreProcess image.


            img_pixels = image.img_to_array(face_detect)
            img_pixels = np.expand_dims(img_pixels, axis = 0)
            img_pixels /= 255
            #face_detect=preprocess_image(face_detect)
            #face_detect/=255
            #cv.imshow('Image',img)
            capture_vector=vgg_face_descriptor.predict(img_pixels)[0,:]
            found=0
            for index in face_dict:
                saved_vector=face_dict[index]
                name=index
                cs=cosine_similarity(capture_vector,saved_vector)
                if (cs<0.45):
                    cv.putText(img, name[5:], (int(x+w+15), int(y-12)), cv.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
                    found=1
                    #cv.rectangle(img,(x,y),(x+w,y+h),(255,0,255),2)
                    break
                    #connect face and text
                cv.line(img,(int((x+x+w)/2),y+15),(x+w,y-20),(255, 0, 0),1)
                cv.line(img,(x+w,y-20),(x+w+10,y-20),(255, 0, 0),1)

                if(found == 0): #if found image is not in our people database
                    cv.putText(img, 'unknown', (int(x+w+15), int(y-12)), cv.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

    cv.imshow('Image',img)
    if cv.waitKey(1) == 13: #13 is the Enter Key
        break
        
capture.release()
cv.destroyAllWindows()